In [1]:
import pandas as pd
import numpy as np
import requests
from lxml import html
import time
from datetime import datetime
from unidecode import unidecode
import warnings
from tqdm import tqdm
import os
import sys

sys.path.append(os.getcwd().replace("\\","/")[:-4])
import models
import functions as func
import paths as ph

sys.path.append(os.getcwd().replace("\\","/") + "/src")
import homologations as ho
import spider

## Ejecucion

In [2]:
# Concatenar sin scrappear
oferts_urls = []
urls_complete = []

RefineAtributes = spider.RefineAtributes
ReadTemp = spider.ReadTemp
TrainModel = spider.TrainModel
models_homologue = ho.models_homologue

print("Bloque 4: concatenado final de archivos")
df = ReadTemp()
df = df[df["estado"] == 1]
print("="*120)

print("Bloque 5: refinando atributos")
df = RefineAtributes(df)
print("-"*120)


print("Correctos final:", df[df["estado"] == 1].shape[0], "|", round((df[df["estado"] == 1].shape[0]/df.shape[0])*100, 2), "%")
print("Dimensión df:", df.shape)
vars_duplicates = ["marca", "modelo", "precio", "año", "motor", "km", "transmision", "tipo_de_carroceria", "tipo_de_combustible", 
                "barrio", "ciudad", "departamento"]
print("Duplicados por atributos:", df[df.duplicated(vars_duplicates)].shape[0])
print("Duplicados por url:", df[df.duplicated(["url"])].shape[0])
print("Duplicados por id:", df[df.duplicated(["id_ofert"])].shape[0])
df = df.drop_duplicates("url").drop_duplicates("id_ofert").drop_duplicates(vars_duplicates)
print("Dimensión df sin duplicados:", df.shape)
print("-"*120)
pendient = set(oferts_urls) - set(df["url"])
print("Final: Registros correctos:", len(urls_complete))
print("Final: Registros con error (excluidos):", len(pendient))
print("="*120)


print("Bloque 6: imputación de datos faltantes en categoricas"); print("-"*50)
df = models.ImputCategorics(df, "marca_modelo", "transmision", 70)
df = models.ImputCategorics(df, "marca_modelo", "tipo_de_carroceria", 70)
df = models.ImputCategorics(df, "marca_modelo", "tipo_de_combustible", 70)
df = models.ImputCategorics(df, "marca_modelo", "puertas", 70)
print("="*120)


# df["transmision"] = df["transmision"].fillna("NULO")
# df["tipo_de_carroceria"] = df["tipo_de_carroceria"].fillna("NULO")
# df["tipo_de_combustible"] = df["tipo_de_combustible"].fillna("NULO")

print("Bloque 7: imputando valores faltantes en cilindraje"); print("-"*50)
df["motor"] = np.log(df["motor"])
df = models.ImputRegression(df, y = 'motor', x = ['año', 'marca', 'modelo', 'tipo_de_combustible', 'transmision', 'tipo_de_carroceria', 
                                                'puertas'], method = "xgboost")
df["motor"] = round(np.exp(df["motor"]), 2)
print("="*120)


print("Bloque 8: criterios de exclusión e inclusión"); print("-"*50)
df = spider.Criterios(df)
print("="*120)

print("Bloque 9: agrupacion de variables"); print("-"*50)
# df = models.ClassicationTreeGroup(df, y = "precio", x = "marca", max_leaf_nodes=[4, 5, 6]); print("-"*50)
df = models.ClassicationTreeGroup(df, y = "precio", x = "modelo", max_leaf_nodes=[25])
print("="*120)

df = df[['estado', 'nombre', 'precio', 'año', 'km', 'km_por_año', 'barrio', 'ciudad','departamento', 'fecha', 'url', 'dtm_etl', 
        'marca_modelo', 'marca', 
        'modelo', 'modelo_agrup', 'version','color', 'tipo_de_combustible', 'puertas', 'transmision', 'motor','tipo_de_carroceria', 
        'ultimo_digito_de_la_placa','con_camara_de_reversa']]

#df.to_csv("df_refine.csv", sep = "|", index = False)
df.to_parquet("data/df_refine.parquet", compression = "gzip")
print("="*120)

print("---------- Precio ----------")
print("Media:", round(df["precio"].mean(), 2))
print("Mediana:", round(df["precio"].median(), 2))
print("CV:", round(np.sqrt(df["precio"].var()) / df["precio"].mean() * 100, 2), "%")
#print("Umbral:", round(umbral_precio, 3))
print("="*120)

print("Bloque 10: entrenando modelo predictivo"); print("-"*50)
model = TrainModel(df, save_model = True)

print("Dimensión final:", df.shape)
print("="*120)

Bloque 4: concatenado final de archivos


AttributeError: module 'paths' has no attribute 'temp'

In [3]:
model.get_params()

{'objective': 'reg:squarederror',
 'base_score': None,
 'booster': 'gbtree',
 'callbacks': None,
 'colsample_bylevel': None,
 'colsample_bynode': None,
 'colsample_bytree': 0.7999999999999999,
 'device': None,
 'early_stopping_rounds': None,
 'enable_categorical': False,
 'eval_metric': None,
 'feature_types': None,
 'gamma': None,
 'grow_policy': None,
 'importance_type': None,
 'interaction_constraints': None,
 'learning_rate': 0.01,
 'max_bin': None,
 'max_cat_threshold': None,
 'max_cat_to_onehot': None,
 'max_delta_step': None,
 'max_depth': 10,
 'max_leaves': None,
 'min_child_weight': 12,
 'missing': nan,
 'monotone_constraints': None,
 'multi_strategy': None,
 'n_estimators': 425,
 'n_jobs': None,
 'num_parallel_tree': None,
 'random_state': None,
 'reg_alpha': None,
 'reg_lambda': None,
 'sampling_method': None,
 'scale_pos_weight': None,
 'subsample': 1,
 'tree_method': None,
 'validate_parameters': None,
 'verbosity': None,
 'alpha': 0.45,
 'eta': 0,
 'lambda': 0.5}

In [4]:
func.pkl.Save(model, f'{ph.models_pkl}/xgboost_{str(actual_date)}.pkl')

NameError: name 'ph' is not defined

# Predict

In [ ]:
año = 2005
km = 150000
motor = 4
marca = "Ford"
modelo = "Explorer"
tipo_de_carroceria = "Camioneta"
puertas = "4_5"
transmision = "Automático"
tipo_de_combustible = "Gasolina"
modelo = df[df["modelo"] == modelo]["modelo_agrup"].unique()[0]

import joblib
files_models = sorted(os.listdir("models_pkl"))[-1]
model = joblib.load("models_pkl/" + files_models)  

cols_numeric = ["precio", "año", "km", "motor", "km_por_año"]
for col in cols_numeric:
    df[col] = df[col].astype(float)
df_train = df[["precio", "año", "km", "motor", "km_por_año", "transmision", "tipo_de_combustible", "tipo_de_carroceria", "puertas",
                "marca", "modelo_agrup"]].dropna()
df_train["precio"] = np.log(df_train["precio"].astype(float))

dim_models = func.pkl.Load("pkl/dim_models_group.pkl")[["modelo", "modelo_agrup"]].drop_duplicates()
dim_doors = func.pkl.Load("pkl/dim_models_group.pkl")[["modelo", "puertas"]].drop_duplicates()

modelo = dim_models[dim_models["modelo"] == modelo]["modelo_agrup"].unique()[0]
puertas = dim_doors[dim_doors["modelo"] == modelo]["puertas"].unique().tolist()

if puertas == []:
    puertas = "4_5"
else:
    puertas = puertas[0]
df_new = pd.DataFrame({"año":[año], "km":[km], "motor":[motor], "km_por_año":[km/(datetime.today().year-año)], 
                        "transmision":[transmision], "tipo_de_combustible":[tipo_de_combustible],"marca":[marca], 
                        "modelo_agrup":[modelo], "puertas":[puertas], "tipo_de_carroceria":[tipo_de_carroceria]})
df_new = pd.get_dummies(df_new, columns = ["transmision", "tipo_de_combustible", "marca", "modelo_agrup",
                                            "puertas", "tipo_de_carroceria"]).replace(True, 1)

df_predict = pd.DataFrame(columns = model.feature_names_in_)
df_predict.loc[0] = 0
features = model.feature_names_in_.tolist()
features_delete = list(set(df_new.columns) - set(features))
if features_delete != []:
    df_predict.drop(columns = features_delete, inplace = True)

predict = int(np.exp(model.predict(features_delete))*1000000)